<a href="https://colab.research.google.com/github/aliash98/Search2Vec/blob/main/Search2Vec_Frequent_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Libraries

In [ ]:
!pip install pandas
!pip install numpy

Importing

In [ ]:
import matplotlib.pyplot as plt
import itertools
from datetime import datetime
import pandas as pd
import numpy as np
import json
import re
import warnings
import gc 
from tqdm import tqdm, trange

# **Preparing Data**

## Reading Data from Drive

In [ ]:
!gdown --id 1W87y-JeBUfyCdsT8uF32rrx0STmfdLYZ
!7z x torob_dataset.7z

Downloading...
From: https://drive.google.com/uc?id=1W87y-JeBUfyCdsT8uF32rrx0STmfdLYZ
To: /content/torob_dataset.7z
295MB [00:05, 57.5MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 295162437 bytes (282 MiB)

Extracting archive: torob_dataset.7z
--
Path = torob_dataset.7z
Type = 7z
Physical Size = 295162437
Headers Size = 315
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      0% 3 - train/base_products.json                                   1% 3 - train/base_products.json                                   2% 3 - train/base_products.json                                 

## Placing into DataFrames

In [ ]:
def read_lowmem(path):
  with open(path) as f:
    res = [json.loads(line) for line in tqdm(f)]
    res = pd.DataFrame(res)
  return res  

product = read_lowmem('train/base_products.json')
click = read_lowmem('train/click_log_train.json') 
search = read_lowmem('train/search_log_train.json') 

# Garbage collector
gc.collect()

1985152it [00:32, 61781.90it/s]
6317934it [00:28, 218552.73it/s]
3694579it [00:44, 82471.60it/s] 


26

In [ ]:
# Checking duplicate query keys
search._id.duplicated().sum()

0

Describing the search dataframe

In [ ]:
search['raw_query'].describe()

count     3694579
unique     116621
top          گوشی
freq        33421
Name: raw_query, dtype: object

# Slicing data

## Retrievinig the most repeated Queries, Clicks and Products

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# most_freq_query_series = search['raw_query'].value_counts().head(100)
# most_freq_query_series = most_freq_query_series.astype(bool)
# most_freq_query_series.head(10)
# search['raw_query'].isin(most_freq_query_series).head(50).sum()
freq_queries = search[search.raw_query.isin(search['raw_query'].value_counts().head(500).index)].copy()
# del most_freq_query_series
del search
gc.collect()
print(freq_queries.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1624870 entries, 0 to 3692754
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   _id        1624870 non-null  object
 1   raw_query  1624870 non-null  object
 2   page       1624870 non-null  int64 
 3   result     1624870 non-null  object
 4   datetime   1624870 non-null  object
dtypes: int64(1), object(4)
memory usage: 74.4+ MB
None


In [ ]:
# Omiting useless columns for saving space
freq_queries = freq_queries.drop("page", axis=1)
freq_queries = freq_queries.drop("result", axis=1)
gc.collect()

50

Dividing into train sample and validation sample 

In [ ]:
freq_queries['datetime'] = [datetime.fromisoformat(d) for d in freq_queries.datetime]
# click['datetime'] = [datetime.fromisoformat(d) for d in click.datetime]

In [ ]:
# The ratio of validation/train size is set to 15/85 aprox ~, 4 days out of 31 days

from_date = datetime.strptime('10/5/2021 00:00:00', '%d/%m/%Y %H:%M:%S')
# to_date = datetime.strptime('13/5/2021 00:00:00', '%d/%m/%Y %H:%M:%S')

freq_queries_validation = freq_queries[[from_date < d  for d in freq_queries.datetime]]
freq_queries_train = freq_queries[[ d <= from_date for d in freq_queries.datetime]]
print(freq_queries_validation.info())
print(freq_queries_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243638 entries, 0 to 572469
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   _id        243638 non-null  object        
 1   raw_query  243638 non-null  object        
 2   datetime   243638 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 7.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1381232 entries, 455198 to 3692754
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   _id        1381232 non-null  object        
 1   raw_query  1381232 non-null  object        
 2   datetime   1381232 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 42.2+ MB
None


In [ ]:
gc.collect()
freq_queries_train.to_csv(r'./gdrive/MyDrive/Colab Notebooks/most_freq_queries_train.csv', index=False)
freq_queries_validation.to_csv(r'./gdrive/MyDrive/Colab Notebooks/most_freq_queries_validation.csv', index=False)

In [ ]:
# finding clicks associated with these queries
freq_query_clicks_train = click[click.search_log_id.isin(freq_queries_train['_id'])].copy()
freq_query_clicks_validation = click[click.search_log_id.isin(freq_queries_validation['_id'])].copy()
print(freq_query_clicks_train.info())
print(freq_query_clicks_validation.info())
del click
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2236735 entries, 779332 to 6311930
Data columns (total 4 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   datetime         object
 1   search_log_id    object
 2   rank             int64 
 3   base_product_id  object
dtypes: int64(1), object(3)
memory usage: 85.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 395070 entries, 0 to 970780
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   datetime         395070 non-null  object
 1   search_log_id    395070 non-null  object
 2   rank             395070 non-null  int64 
 3   base_product_id  395070 non-null  object
dtypes: int64(1), object(3)
memory usage: 15.1+ MB
None


14

In [ ]:
freq_query_clicks_train.to_csv(r'./gdrive/MyDrive/Colab Notebooks/most_freq_clicks_train.csv', index=False)
freq_query_clicks_validation.to_csv(r'./gdrive/MyDrive/Colab Notebooks/most_freq_clicks_validation.csv', index=False)

In [ ]:
freq_products_train = product[product._id.isin(freq_query_clicks_train['base_product_id'])].copy()
freq_products_validation = product[product._id.isin(freq_query_clicks_validation['base_product_id'])].copy()
print(freq_products_train.info())
print(freq_products_validation.info())
del product
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108956 entries, 15 to 1985139
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   _id            108956 non-null  object
 1   category_name  108956 non-null  object
 2   sellers        108956 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 45544 entries, 26 to 1985131
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id            45544 non-null  object
 1   category_name  45544 non-null  object
 2   sellers        45544 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB
None


305

In [ ]:
freq_products_train.to_csv(r'./gdrive/MyDrive/Colab Notebooks/most_freq_products_train.csv', index=False)
freq_products_validation.to_csv(r'./gdrive/MyDrive/Colab Notebooks/most_freq_products_validation.csv', index=False)

In [ ]:
gc.collect()

88